In [1]:
#Imports
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd                 
import matplotlib.pyplot as plt    
import numpy as np 

In [2]:
df_dados = pd.read_csv('credito.csv', sep = ';')

In [3]:
#Análise exploratório para verificar colunas TARGET, REMOVER, TRANSFORMA TEXTO EM NÚMERO
#ATENÇÃO POIS O NOME DO CAMPO APARECE APÓS A RELAÇÃO DE VALORES

#TARGET:
#AtrasoPrimeiroPagamento

#REMOVER:
#'Cliente_Id','Cpf','Nome','Bairro','Cidade','Uf','Fazendeiro'

#TRANSFORMAR TEXTO EM NUMERO:

#'EstadoCivil','Sexo','Escolaridade','TipoMoradia','Filho','Classe','Trabalha','TempoTrabalho','TempoTrabalho','Nacionalidade',
#'SpcPrimeiraConsultaScore','SpcPrimeiraConsultaRestricao','Nascimento','PrimeiraCompra','UltimaCompra','IdadePrimeiraCompra',
#'Renda','Risco','Qualidade','Satisfacao','Renegociacao','LimiteCredito','Funcionario','AbertoSituacao','Cargo','SpcHojeRestricao',
#VERIFICA PELO VALORES DISTINCT DE CADA CAMPO COMO PROCEDER
for x in df_dados.columns:
    print(df_dados[x].drop_duplicates())

0        33
1        34
2        68
3        85
4       126
       ... 
995    8992
996    9002
997    9003
998    9019
999    9020
Name: Cliente_Id, Length: 1000, dtype: int64
0       270246606
1       271026618
2       408843378
3       502612809
4       862450667
          ...    
995    5734462616
996    5740608635
997    5741516645
998    5752791685
999    5753111696
Name: Cpf, Length: 1000, dtype: int64
0      MARIA APARECIDA DA CRUZ 639280     
1      CELIA LEONEL DA S. OLIVEIRA 805603 
2      SIMONY CRISTINA DA SILVA           
3      MARIA ROSANGELICA BRANDÃO GUSMÃO   
4      fatima martins de sousa622323      
                      ...                 
995    NILDA GONZAGA DE OLIVEIRA          
996    MARCELA APARECIDA FERREIRA         
997    LIRIA MENDES FERREIRA-804437       
998    ALCIMAR FERREIRA ALVES 804300      
999    AMILTON NOGUEIRA JUNIOR-805545     
Name: Nome, Length: 1000, dtype: object
0      EDUARDO GIRAUDON                              ...
1      JD DO BOSQ

0         0.0
1       900.0
5       500.0
12      600.0
13      563.0
        ...  
912     570.0
941    1225.0
947    1875.0
978     720.0
986    2970.0
Name: LimiteCredito_Ind, Length: 91, dtype: float64
0       62
1       46
2      120
4       45
5       44
      ... 
409     81
555     82
569     84
598     69
884      6
Name: IdadeAtual_Ind, Length: 65, dtype: int64
0       57
1       41
2      113
4       39
5       34
      ... 
409     75
519     63
569     77
640     72
884      0
Name: IdadePrimeiraCompra_Ind, Length: 65, dtype: int64
0      1200.0
1      1245.0
2         0.0
4      1442.0
5       880.0
        ...  
894    1037.0
916    1286.0
952    1299.0
973    1737.0
993     920.0
Name: Renda_Ind, Length: 156, dtype: float64
0       16
1        0
2        1
3        3
4        5
      ... 
812    310
826     64
853    261
923    224
934     43
Name: PrazoMediaAtraso, Length: 95, dtype: int64
0      2
2      6
3      3
5      4
10     1
27     5
78     7
101    8
Name: Pr

In [4]:
#Pré-processamento 
#Feature selection 
#Removendo campos identificadores
df_dados = df_dados.drop(['Cliente_Id', 'Cpf', 'Nome', 'Bairro', 'Cidade', 'Uf', 'Fazendeiro'], axis=1)

In [5]:
df_dados.head()

,EstadoCivil,Sexo,Escolaridade,TipoMoradia,Filho,Classe,Trabalha,TempoTrabalho,SpcHojeRestricao,SpcPrimeiraConsultaScore,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
0,Casado,Feminino,Fundamental-Incompleto,Própria,Sim,PENSIONISTA,Trabalha,18 ANOS 5 MESES,Com Restrição,NaN,...,380.0,0.0,62,57,1200.0,16,2,0.0,0.0,0.0
1,Casado,Feminino,Superior-Completo,Própria,Sim,INDUSTRIARIOS/ OBRA,Trabalha,7 ANOS 5 MESES,Sem Restrição,NaN,...,200.0,900.0,46,41,1245.0,0,2,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 ANOS 5 MESES,Sem Restrição,NaN,...,0.0,0.0,120,113,0.0,1,6,NaN,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 ANOS,Sem Restrição,NaN,...,0.0,0.0,120,113,0.0,3,3,NaN,0.0,0.0
4,Viúvo,Feminino,Fundamental-Completo,Própria,Sim,PENSIONISTA,Trabalha,8 ANOS,Com Restrição,NaN,...,500.0,0.0,45,39,1442.0,5,3,0.0,0.0,0.0


In [6]:
#colunas categóricas que terão valores por categoria convertidas em número (talvez reavaliar campos data)
#Cria dicionário com colunas categóricas qcolunas categóricas que terão valores por categoria convertidas em número 
colunas_texto = ['EstadoCivil','Sexo','Escolaridade','TipoMoradia','Filho','Classe','Trabalha','TempoTrabalho','TempoTrabalho','SpcPrimeiraConsultaScore','SpcPrimeiraConsultaRestricao','Nascimento','PrimeiraCompra','UltimaCompra','IdadePrimeiraCompra','Renda','Risco','Qualidade','Satisfacao','Renegociacao','LimiteCredito','Funcionario','AbertoSituacao','Cargo','SpcHojeRestricao','Nacionalidade']
colunas_texto

['EstadoCivil',
 'Sexo',
 'Escolaridade',
 'TipoMoradia',
 'Filho',
 'Classe',
 'Trabalha',
 'TempoTrabalho',
 'TempoTrabalho',
 'SpcPrimeiraConsultaScore',
 'SpcPrimeiraConsultaRestricao',
 'Nascimento',
 'PrimeiraCompra',
 'UltimaCompra',
 'IdadePrimeiraCompra',
 'Renda',
 'Risco',
 'Qualidade',
 'Satisfacao',
 'Renegociacao',
 'LimiteCredito',
 'Funcionario',
 'AbertoSituacao',
 'Cargo',
 'SpcHojeRestricao',
 'Nacionalidade']

In [7]:
#loop que percorre lista de colunas categóricas que terão valores por categoria convertidas em número 
novo_valor_numerico = 0
i = 0
for x in colunas_texto:
    for y in df_dados[x].drop_duplicates():
        df_dados[x].replace({y: str(i)}, inplace=True)
        i+=1
    i = 0

In [8]:
#Trantando dados missing
df_dados.isnull()

,EstadoCivil,Sexo,Escolaridade,TipoMoradia,Filho,Classe,Trabalha,TempoTrabalho,SpcHojeRestricao,SpcPrimeiraConsultaScore,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [9]:
#Se os valores missing mesmos substituídos estiverem influenciando a análise, experimetar df_dados.dropna() ou 
#df_dados.interpolate(method ='linear', limit_direction ='forward') 
df_dados = df_dados.replace(to_replace = np.nan, value = -99) 

In [10]:
df_dados.head()

,EstadoCivil,Sexo,Escolaridade,TipoMoradia,Filho,Classe,Trabalha,TempoTrabalho,SpcHojeRestricao,SpcPrimeiraConsultaScore,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
0,0,0,0,0,0,0,0,0,0,0,...,380.0,0.0,62,57,1200.0,16,2,0.0,0.0,0.0
1,0,0,1,0,0,1,0,1,1,0,...,200.0,900.0,46,41,1245.0,0,2,0.0,0.0,0.0
2,1,1,2,1,1,2,1,2,1,0,...,0.0,0.0,120,113,0.0,1,6,-99.0,0.0,0.0
3,1,1,2,1,1,2,1,3,1,0,...,0.0,0.0,120,113,0.0,3,3,-99.0,0.0,0.0
4,2,0,3,0,0,0,0,4,0,0,...,500.0,0.0,45,39,1442.0,5,3,0.0,0.0,0.0


In [11]:
# Visualizando a correlação em tabela
# Coeficiente de correlação: 
# +1  = forte correlação positiva
# 0   = não há correlação
# -1  = forte correlação negativa
df_dados.corr()

,SpcPrimeiraConsultaQtdConsulta,SpcPrimeiraConsultaSpcQtdRestricao,AtrasoPrimeiroPagamento,VendaVlrVenda,VendaQtdVenda,VendaVlrPago,VendaVlrAtraso,VendaVlrAVencer,PagamentoVlrPago,PagamentoQtdPago,...,PrimeiroLimiteCredito_Ind,LimiteCredito_Ind,IdadeAtual_Ind,IdadePrimeiraCompra_Ind,Renda_Ind,PrazoMediaAtraso,PrazoMedioVenda,VlrDh,VlrCartaoCredito,VlrCartaoDebito
SpcPrimeiraConsultaQtdConsulta,1.000000,0.255704,0.049548,0.065085,0.012953,0.065268,-0.025431,0.069882,0.065268,0.023822,...,0.094383,0.147296,-0.042844,-0.030463,0.049071,-0.020870,-0.063970,0.087751,-0.000028,0.029597
SpcPrimeiraConsultaSpcQtdRestricao,0.255704,1.000000,0.049527,-0.001867,0.019971,-0.009400,0.056962,-0.004383,-0.009400,-0.005816,...,0.003053,-0.057826,0.009693,0.012742,0.028760,0.002045,-0.022458,0.039482,-0.010100,-0.005721
AtrasoPrimeiroPagamento,0.049548,0.049527,1.000000,0.106308,0.068214,0.060760,0.305913,0.091118,0.060760,0.056234,...,-0.067494,-0.137608,-0.058739,-0.060484,0.004299,0.112859,0.161344,-0.011068,0.034242,-0.006092
VendaVlrVenda,0.065085,-0.001867,0.106308,1.000000,0.662898,0.988855,0.057061,0.479023,0.988855,0.848791,...,0.369203,0.371118,-0.034136,-0.037607,0.013940,-0.053679,-0.062470,0.489417,0.257018,0.249584
VendaQtdVenda,0.012953,0.019971,0.068214,0.662898,1.000000,0.674849,-0.053409,0.232181,0.674849,0.774818,...,0.137218,0.225567,-0.055383,-0.067485,0.029822,-0.035917,-0.165980,0.258136,0.385780,0.453235
VendaVlrPago,0.065268,-0.009400,0.060760,0.988855,0.674849,1.000000,-0.074822,0.421693,1.000000,0.868549,...,0.361026,0.373797,-0.026164,-0.029600,0.017705,-0.046920,-0.097603,0.455936,0.263715,0.258644
VendaVlrAtraso,-0.025431,0.056962,0.305913,0.057061,-0.053409,-0.074822,1.000000,-0.042206,-0.074822,-0.079454,...,-0.049012,-0.213015,-0.058691,-0.061954,-0.032150,-0.054126,0.246435,-0.079306,-0.030495,-0.026775
VendaVlrAVencer,0.069882,-0.004383,0.091118,0.479023,0.232181,0.421693,-0.042206,1.000000,0.421693,0.260516,...,0.348151,0.494911,-0.013885,-0.010244,0.014206,-0.015095,-0.010915,0.769663,0.080997,0.041774
PagamentoVlrPago,0.065268,-0.009400,0.060760,0.988855,0.674849,1.000000,-0.074822,0.421693,1.000000,0.868549,...,0.361026,0.373797,-0.026164,-0.029600,0.017705,-0.046920,-0.097603,0.455936,0.263715,0.258644
PagamentoQtdPago,0.023822,-0.005816,0.056234,0.848791,0.774818,0.868549,-0.079454,0.260516,0.868549,1.000000,...,0.197845,0.244404,-0.020372,-0.026054,0.002112,-0.048255,-0.051865,0.269292,0.283317,0.440268


In [13]:
# Visualizando correlação com variavel target
corr_matrix = df_dados.corr().abs()
print(corr_matrix['AtrasoPrimeiroPagamento'].sort_values(ascending=False).head(100))

AtrasoPrimeiroPagamento               1.000000
AbertoQtd                             0.336983
AbertoVlr                             0.316088
AbertoQtdAtraso                       0.308405
AbertoVlrAtraso                       0.305913
VendaVlrAtraso                        0.305913
AbertoQtdAtrasoAcima                  0.304577
AbertoVlrAtrasoAcima                  0.301893
AbertoQtdPrevPerda                    0.279902
AbertoVlrPrevPerda                    0.278277
PrazoMedioVenda                       0.161344
PagamentoVlrAtraso30                  0.158858
PagamentoQtdAtraso60                  0.156866
PagamentoQtdAtraso30                  0.139490
LimiteCredito_Ind                     0.137608
PagamentoVlrAtraso60                  0.132433
AbertoQtdAVencer                      0.118258
PrazoMediaAtraso                      0.112859
AbertoQtdCano                         0.110036
VendaVlrVenda                         0.106308
AbertoVlrCano                         0.105224
PagamentoQtdA

In [14]:
#Alta correlação ou colineridade pode causar overfitting
# Índice de features com alta correlação
#nesse caso rever o fit e ajustar conforme resultados dos treinos e aumento do volume de dados após análise da amostra
to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column] > 0.50)]
print('Columns to drop: ' , (len(to_drop)))

Columns to drop:  51


In [15]:
#Pré-processamento 
#Redução de dimensionalidade
# PCA - mantém 90% da variancia
pca = PCA(0.90)
principal_components = pca.fit_transform(df_dados)
principal_df = pd.DataFrame(data = principal_components)
print(principal_df.shape)
#somente uma feature explica a variância dos dados, entender melhor e ver se c volume muda o comportamento

(1000, 1)


In [16]:
#Split variáveis preditoras e target
df_dados_target = df_dados['AtrasoPrimeiroPagamento']
df_dados = df_dados.drop(['AtrasoPrimeiroPagamento'], axis=1)

In [17]:
#Pré-processamento 
#Normalização
#Colocando os dados na mesma escala
scaler = StandardScaler()

In [18]:
#virgulas foram substituídas por ponto no arquivo fonte antes da importação para agilizar(fazer por aqui)
nparray_dados = scaler.fit_transform(df_dados)
nparray_dados

array([[-0.94636022, -0.52880427, -1.74900954, ...,  0.30952574,
        -0.08963013, -0.06761548],
       [-0.94636022, -0.52880427, -1.12967188, ...,  0.30952574,
        -0.08963013, -0.06761548],
       [-0.2827135 ,  0.75782412, -0.51033423, ..., -0.44143789,
        -0.08963013, -0.06761548],
       ...,
       [-0.94636022, -0.52880427,  0.72834108, ...,  0.30952574,
        -0.08963013, -0.06761548],
       [ 1.04457994,  2.04445252,  0.10900343, ..., -0.44143789,
        -0.08963013, -0.06761548],
       [ 1.04457994,  2.04445252,  0.72834108, ...,  0.30952574,
        -0.08963013, -0.06761548]])

In [19]:
#Criação dos modelos
# Definindo a taxa de split
split_test_size = 0.30
# Criando dados de treino e de teste
x_treino, x_teste, y_treino, y_teste = train_test_split(df_dados, df_dados_target, test_size = split_test_size, random_state = 42)

In [20]:
modelo = RandomForestClassifier(random_state = 42)
modelo.fit(x_treino, y_treino.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [21]:
# Verificando os dados de treino
rf_predict_train = modelo.predict(x_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_treino, rf_predict_train)))

Exatidão (Accuracy): 1.0000


In [22]:
# Verificando nos dados de teste
rf_predict_test = modelo.predict(x_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_teste, rf_predict_test)))
print()

Exatidão (Accuracy): 0.8067



In [23]:
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(y_teste, rf_predict_test, labels = [1, 0])))
print("")

#F1 Score: quanto maior melhor

print("Classification Report")
print(metrics.classification_report(y_teste, rf_predict_test, labels = [1, 0]))

Confusion Matrix
[[156  33]
 [ 25  86]]

Classification Report
              precision    recall  f1-score   support

           1       0.86      0.83      0.84       189
           0       0.72      0.77      0.75       111

    accuracy                           0.81       300
   macro avg       0.79      0.80      0.80       300
weighted avg       0.81      0.81      0.81       300



In [24]:
# Versão do modelo usando Regressão Logística
modelo_v2 = LogisticRegression(C = 0.7, random_state = 42)
modelo_v2.fit(x_treino, y_treino.ravel())
lr_predict_test = modelo_v2.predict(x_teste)

C:\Users\diogo\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [25]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(y_teste, lr_predict_test)))
print()
print("Classification Report")
print(metrics.classification_report(y_teste, lr_predict_test, labels = [1, 0]))

Exatidão (Accuracy): 0.7600

Classification Report
              precision    recall  f1-score   support

           1       0.82      0.79      0.81       189
           0       0.66      0.71      0.69       111

    accuracy                           0.76       300
   macro avg       0.74      0.75      0.75       300
weighted avg       0.76      0.76      0.76       300



In [ ]:
lr_predict_test

In [ ]:
#PAREI AQUI TARDE DE 12/03/2020
#CARREGAR NOVOS DADOS COM MAIS VOLUME E OTIMIZAR POR MEIO DE ESCOLHA AUTOMÁTICA DE PARAMETROS E MODELOS
#USER GRID SEARCH
#sklearn.model_selection.GridSearchCV

In [ ]:
#MODELO PARA TREINAR VÁRIOS MODELOS E VER QUAL É O MELHOR MODELO
#sklearn.ensemble.GradientBoostingClassifier